In [4]:
from __future__ import print_function, division
from thinkbayes2 import Pmf, Suite
import thinkplot

# Report 1
#### Matthew Beaudouin-Lafon

## Cookie Problem ++
#### From Think Bayes Chapter 2
With no replacement

In [5]:
class Cookies(Suite):
    """
    Bayesian model for the cookie problem. Extends Suite.
    """
    mixes = {'B1' :
                {'vanilla' : 30,
                'chocolate' : 10},
            'B2' : 
                {'vanilla' : 20,
                'chocolate' : 20}}
    def Likelihood(self, data, hypo):
        """
        parameters:
          -  data (either 'chocolate' or 'vanilla')
          -  hypo (hypothesis, either 'B1' or 'B2')
          
        returns: Likelihood of the data under the hypothesis
        """
        prob = self.mixes[hypo][data]
        
        totalCookies = 0
        for flavor in self.mixes[hypo]:
            totalCookies += self.mixes[hypo][flavor]
            
        if self.mixes[hypo][data] != 0:
            self.mixes[hypo][data] -= 1
            totalCookies -= 1
        
        return prob / totalCookies
    
cookies = Cookies(['B1', 'B2'])

cookies.Update('chocolate') 
print(cookies.mixes)
cookies.Print()

cookies.Update('chocolate') 
print(cookies.mixes)
cookies.Print()

# cookies.Update('chocolate') # This makes things break but I don't understand why.
# print(cookies.mixes)        # When taking a non existing chocolate cookie, the probability should stick to 0.
# cookies.Print()

{'B1': {'vanilla': 3, 'chocolate': 0}, 'B2': {'vanilla': 2, 'chocolate': 1}}
B1 0.333333333333
B2 0.666666666667
{'B1': {'vanilla': 3, 'chocolate': 0}, 'B2': {'vanilla': 2, 'chocolate': 0}}
B1 0.0
B2 1.0


## Elvis Problem
#### From David MacKay's "Information Theory, Inference, and Learning Algorithms"

Elvis Presley had a twin brother who died at birth.  What is the probability that Elvis was an identical twin?

To answer this one, you need some background information: According to the Wikipedia article on twins:  ``Twins are estimated to be approximately 1.9% of the world population, with monozygotic twins making up 0.2% of the total---and 8% of all twins.''

In [7]:
class Pelvis(Suite):
    """
    Bayesian model for the Elvis problem. Extends Suite.
    """
    def Likelihood(self, data, hypo):
        """parameters:
          -  data (either 'brother' or 'sister')
          -  hypo (hypothesis, either 'monozygotic twin' or 'boring twin')
          
        returns: Likelihood of the data under the hypothesis"""
        if hypo == 'monozygotic twin':
            if data == 'brother':
                return 1
            else:       # data == 'sister'
                return 0
        else: # if hypo == 'boring twin':
            if data == 'brother':
                return 0.5
            else:       # data == 'sister'
                return 0.5

In [8]:
elvis = Pelvis(['monozygotic twin', 'boring twin'])
elvis['monozygotic twin'] *= 8
elvis['boring twin'] *= 92

for i in range(3):          # I'm glad this is just hypothetical
    elvis.Update('brother')

#elvis.Update('sister')    
elvis.Print()

boring twin 0.589743589744
monozygotic twin 0.410256410256


In order to understand the Bayesian dynamics, I wonder: how many twins would Elvis need to have to have a 90% confidence in the monozygotic hypothesis?

In [9]:
elvis = Pelvis(['monozygotic twin', 'boring twin'])
elvis['monozygotic twin'] *= 8
elvis['boring twin'] *= 92
elvis.Normalize()

numBrothers = 0
while elvis['monozygotic twin'] < 0.9:          # I'm glad this is just hypothetical
    elvis.Update('brother')
    numBrothers += 1

#elvis.Update('sister') 
elvis.Print()
print('It took ' + str(numBrothers) + ' brothers')

boring twin 0.0824372759857
monozygotic twin 0.917562724014
It took 7 brothers


Notes:
    Even with four twin brothers, it's not that much more likely that Elvis was monozygotic (58%). It takes 7 twin brothers to have a 90% confidence in the monozygotic hypothesis.
    This while loop is also an interesting way to test how much data you would need to make a decision.
    Also, throwing a sister in the loop makes the hypotheses 0 and 1 for monozygotic and boring twins respectively, which is exactly what you would expect

## Smoker Problem
#### From http://allendowney.blogspot.com/2011/10/my-favorite-bayess-theorem-problems.html

According to the CDC, ``Compared to nonsmokers, men who smoke are about 23 times more likely to develop lung cancer and women who smoke are about 13 times more likely.''
If you learn that a woman has been diagnosed with lung cancer, and you know nothing else about her, what is the probability that she is a smoker?

According to http://kff.org/other/state-indicator/smoking-adults-by-gender/?currentTimeframe=0&selectedDistributions=female&selectedRows=%7B%22wrapups%22:%7B%22united-states%22:%7B%7D%7D%7D&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D
15.3% of women smoke in the US.

According to http://www.tabac-info-service.fr/Vos-questions-Nos-reponses/Chiffres-du-tabac
28% of women smoke in France.
36% of men in France. (sigh)

In [17]:
class Smokers(Suite):
    """
    Bayesian model for the smokers problem. Extends Suite.
    """
    def Likelihood(self, data, hypo):
        """
        parameters:
          -  data (either 'cancer' or 'no cancer')
          -  hypo (hypothesis, either 'smoker' or 'non smoker')
          
        returns: Likelihood of the data under the hypothesis
        """
        if hypo == 'smoker':
            if data == 'cancer':
                return 13
            else: # no cancer
                return 1
        else: #Not smoker
            return 1
        
smokers = Smokers(['smoker', 'non-smoker'])
smokers['smoker'] *= 28
smokers['non-smoker'] *= 72

smokers.Update('cancer')

smokers.Print()

non-smoker 0.165137614679
smoker 0.834862385321


In [18]:
# What if we only know it's a person with cancer? We can hypothesize whether it's a man or a woman.

class Smokers2(Suite):
    """
    Bayesian model for the smokers problem. Extends Suite.
    """
    def Likelihood(self, data, hypo):
        """
        parameters:
          -  data (either 'cancer' or 'no cancer')
          -  hypo (hypothesis, either 'smoker' or 'non smoker')
          
        returns: Likelihood of the data under the hypothesis
        """
        if data == 'cancer':
            like = {'SM' : 23, 'SW' : 13, 'NM' : 1, 'NW' : 1}
            return like[hypo]
        else:
            return 1

proportionOfWomen = 0.28
proportionOfMen = 0.36
        
smokers = Smokers2(['SM', 'SW', 'NM', 'NW']) # Smoker (Man, Woman), non-Smoker(Man, Woman)
# The actual portion of men who smoke is proportionOfMen/2, but since that factor is applied to all hypotheses, 
# normalization will take care of it
smokers['SM'] *= proportionOfMen
smokers['SW'] *= proportionOfWomen
smokers['NM'] *= 1 - proportionOfMen
smokers['NW'] *= 1 - proportionOfWomen

smokers.Update('cancer')
smokers.Print()


NM 0.0481927710843
NW 0.0542168674699
SM 0.623493975904
SW 0.274096385542


Not knowing the probability of getting lung cancer slightly threw me off in terms of implementing the code for it; I was getting tripped up by the extra variable lying around.
Doing it by hand made me realize how it cancelled out through normalizing, so that helped. It also reminded me being introduced to Bayes Theorem as 'flipping the probability tree', which is basically what this problem is about.

![Table based solution of the Smoker Problem](Images/smokerProblem.JPG)

## Card Problem
#### From yours truly

I have taken a certain number of cards from a deck of cards with no duplicates. You can take a card, look at it (and remember it), and place it back randomly.
How many cards are left?

In [19]:
import random

In [54]:
# Since we don't care about suits or values, we can simply number the cards 0 through 51.
# This will allow us to keep track of repeats.
# Here's the setup:
# There are 52 hypotheses, 0 through 51, the nth of which states that there are n cards left

class Cards(Suite):
    """
    Bayesian model for the Card problem. Extends Suite.
    """
    seenCards = {}
    numSeenCards = 0
    
    def Likelihood(self, data, hypo):     # This problem is a slight variation of the tank problem (Think Bayes Chapter 3)
        """
        parameters:
          -  data (card that has been seen, which is a number between 0 and 51)
          -  hypo (hypothesis, number between 0 and 51 representing the hypothetical number of cards left in the deck)
          
        returns: Likelihood of the data under the hypothesis
        """
        if 0 < self.seenCards.get(data):  # So the likelihood function is very similar.
            self.seenCards[data] += 1
        else:
            self.numSeenCards += 1
            self.seenCards[data] = 1
        
        if hypo < self.numSeenCards:      # So if the hypothesis states that there are n cards left, and we've seen more than n cards
            return 0                      # That hypothesis is nullified
        else:
            return (1 / hypo)
        
# Setup the premise by taking away 3 cards
deck = range(52)
for removedCard in [13, 9, 46]:
    del deck[removedCard]

cards = Cards(range(52))
cards.deck = deck

seenCards = [] # This doesn't do anything Bayes-ially speaking; it's just to understand which cards we've seen
numTrials = 3

for i in range(numTrials):
    # Find a random card
    positionInDeck = random.randint(0, len(deck)-1) 
    seenCards.append(deck[positionInDeck])
    # And update the model
    cards.Update(deck[positionInDeck])

print('Cards seen:')
print(seenCards)
print('You probably took ' + str(52 - cards.Mean()) + ' cards out')

Cards seen:
[32, 19, 38]
You probably took 47.1148040987 cards out


This is just a tank problem in disguise (more of a thin veil really). The slight trickiness is that either you need to think of it as how many cards are left, or adapt the likelihood function to take it into account. 

It took me a while to figure this one out, because I was hooked on thinking about how much information you get from seeing a duplicate (which explains why I'm keeping track of more than just the cards I've seen in that dictionary). 
However, looking at this solution and a few examples, I realize that the duplicity of cards is absolutely where the information is at. When you start seeing cards over and over, that means you've exhausted the possible cards, so the Update() process is getting closer to an answer.
So maybe not the coolest problem, but it helped me get a firmer grasp of how Bayesian dynamics.

# Binomial and hypergeometric distributions
Class work

Suppose you have an urn that contains 30 red and 60 blue marbles.  If you draw 5 marbles at once (with, then without putting them back) what is the probability that 3 are blue?

In [21]:
from scipy.special import binom
import scipy.stats

print(scipy.stats.binom.pmf(3, 5, 2.0/3))
print(scipy.stats.hypergeom.pmf(3, 90, 60, 5))

0.329218106996
0.338701886912


Random question: 
Can you measure wether you have enough data by looking at the deviation in results from different sets of priors?
What two sets of priors oppose each other as much as possible without touching 0?